## Imports

In [4]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
import os
import subprocess

import math

import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_model_optimization as tfmot

## Environment Variables

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="6"
os.environ['TF_DETERMINISTIC_OPS'] = '1'

## Basic Statistics

Insert path files appropriately for data which can be found after generation in results folder for this experiment.

In [ ]:
net = "res" # res -> ResNet, dense -> DenseNet, mobile -> MobileNet

In [ ]:
c = "1" # Hyperparameter C value from data
locald = './results/pqat/DIVA/'+net+'net/' # Please change to the corresponding results folder for evaluation
folderName= net + 'net_imagenet_images_second'
filterName= net +'net_imagenet_filters_second'
dataFolder= net +'net_imagenet_data_second'
dataName= 'second'

In [ ]:
def print_stats():
    advist_data = np.array([])

    with open(locald + dataFolder+"/"+dataName+'_advdist_data.csv', 'r') as filehandle:
        for line in filehandle:
            temp_data = line[:-2]

        advist_data = np.concatenate((advist_data, np.array(temp_data.split(", "))))
        
    steps_data = np.array([])

    with open(locald +dataFolder+"/"+dataName+'_steps_data.csv', 'r') as filehandle:
        for line in filehandle:
            temp_data = line[:-2]

        steps_data = np.concatenate((steps_data, np.array(temp_data.split(", "))))
    
    time_data = np.array([])

    with open(locald +dataFolder+"/"+dataName+'_time_data.csv', 'r') as filehandle:
        for line in filehandle:
            temp_data = line[:-2]

        time_data = np.concatenate((time_data, np.array(temp_data.split(", "))))
    
    advist_datak = np.array([])

    with open(locald +dataFolder+"/"+dataName+'_advdistk_data.csv', 'r') as filehandle:
        for line in filehandle:
            temp_data = line[:-2]

        advist_datak = np.concatenate((advist_datak, np.array(temp_data.split(", "))))
        
    steps_datak = np.array([])

    with open(locald +dataFolder+"/"+dataName+'_stepsk_data.csv', 'r') as filehandle:
        for line in filehandle:
            temp_data = line[:-2]

        steps_datak = np.concatenate((steps_datak, np.array(temp_data.split(", "))))
    
    time_datak = np.array([])

    with open(locald +dataFolder+"/"+dataName+'_timek_data.csv', 'r') as filehandle:
        for line in filehandle:
            temp_data = line[:-2]

        time_datak = np.concatenate((time_datak, np.array(temp_data.split(", "))))
    
    time_data = time_data.astype('float')
    advist_data = advist_data.astype('float')
    steps_data = steps_data.astype('float')

    time_datak = time_datak.astype('float')
    advist_datak = advist_datak.astype('float')
    steps_datak = steps_datak.astype('float')

    time_data_ = np.mean(time_data), np.std(time_data)
    advdist_data_ = np.mean(advist_data), np.std(advist_data)
    steps_data_ = np.mean(steps_data), np.std(steps_data)

    time_datak_ = np.mean(time_datak), np.std(time_datak)
    advdist_datak_ = np.mean(advist_datak), np.std(advist_datak)
    steps_datak_ = np.mean(steps_datak), np.std(steps_datak)
    
    print("Number of Successes",len(steps_data))
    print("Total Time",np.sum(time_data)) 
    print("Total Steps",np.sum(steps_data))
    print("Steps",steps_data_)
    print("Time", time_data_)
    
    print()
    print("Time5",time_datak_) 
    print("Dist5",advdist_datak_) 
    print("Steps5",steps_datak_)
    
    print()
    print("Number of top1 success",(len(steps_data))/3000)
    print("Number of top5 success",len(steps_datak)/3000)


In [ ]:
print("c", c)
print_stats()

## Load Models

In [ ]:
'''
Custom layers for DenseNet to support quantisation
'''

class DefaultBNQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):

    def get_weights_and_quantizers(self, layer):
        return []

    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass
    def set_quantize_activations(self, layer, quantize_activations):
        pass
    def get_output_quantizers(self, layer):
        return [tfmot.quantization.keras.quantizers.MovingAverageQuantizer(num_bits=8, per_axis=False, symmetric=False, narrow_range=False)]

    def get_config(self):
        return {}
    
    
class NoOpQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    """Use this config object if the layer has nothing to be quantized for 
    quantization aware training."""

    def get_weights_and_quantizers(self, layer):
        return []

    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass

    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        # Does not quantize output, since we return an empty list.
        return []

    def get_config(self):
        return {}
    
    
def apply_quantization(layer):
    if 'bn'  in layer.name:
        return tfmot.quantization.keras.quantize_annotate_layer(layer,DefaultBNQuantizeConfig())
    elif 'concat' in layer.name:
        return tfmot.quantization.keras.quantize_annotate_layer(layer,NoOpQuantizeConfig())
    else:
        return tfmot.quantization.keras.quantize_annotate_layer(layer)

In [ ]:
# input image dimensions
img_rows, img_cols = 224 ,224
input_shape = (img_rows, img_cols, 3)

In [ ]:
#Choice of model to evaluate
model_type = "m" # r -> ResNet, d -> DenseNet, m -> MobileNet

In [ ]:
if model_type == "r":
    model_ = ResNet50(input_shape=input_shape)
    pqat_model = tfmot.quantization.keras.quantize_model(model_)
    p_model = ResNet50(input_tensor = pqat_model.input)
    model = ResNet50(input_tensor = pqat_model.input)
    model.load_weights("../weights/fp_model_40_resnet50.h5")
    p_model.load_weights("../weights/p_model_40_resnet50.h5")
    pqat_model.load_weights("../weights/pqat_model_40_resnet50.h5")
    print("ResNet Done")
    
elif model_type == "m":
    model_ = MobileNet(input_shape=input_shape)
    pqat_model = tfmot.quantization.keras.quantize_model(model_)
    p_model = MobileNet(input_tensor = pqat_model.input)
    model = ResNet50(input_tensor = pqat_model.input)
    model.load_weights("../weights/fp_model_40_mobilenet.h5")
    p_model.load_weights("../weights/p_model_40_mobilenet.h5")
    pqat_model.load_weights("../weights/pqat_model_40_mobilenet.h5")
    print("MobNet Done")
    
else:
    model_ = tf.keras.applications.DenseNet121(input_shape=(img_rows, img_cols,3))
    # Create a base model
    base_model = model_
    # Helper function uses `quantize_annotate_layer` to annotate that only the 
    # Dense layers should be quantized.

    LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
    MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

    # Use `tf.keras.models.clone_model` to apply `apply_quantization_to_dense` 
    # to the layers of the model.
    annotated_model = tf.keras.models.clone_model(
        base_model,
        clone_function=apply_quantization,
    )

    with tfmot.quantization.keras.quantize_scope({'DefaultBNQuantizeConfig': DefaultBNQuantizeConfig, 'NoOpQuantizeConfig': NoOpQuantizeConfig}):
        pqat_model = tfmot.quantization.keras.quantize_apply(annotated_model)

    model = DenseNet121(input_tensor = pqat_model.input)
    p_model = DenseNet121(input_tensor = pqat_model.input)
    model.load_weights("../weights/fp_model_40_densenet121.h5")
    pqat_model.load_weights("../weights/pqat_model_40_densenet121.h5")
    p_model.load_weights("../weights/p_model_40_densenet121.h5")

    preprocess = tf.keras.applications.densenet.preprocess_input
    decode = tf.keras.applications.densenet.decode_predictions
    net = 'dense'

In [ ]:
model.trainable = False
p_model.trainable = False
pqat_model.trainable = False
model.compile()
p_model.compile()
pqat_model.compile()

In [ ]:
#Appropriate Preprocessing and Decoding predictions
if model_type=="r":
    preproc = tf.keras.applications.resnet.preprocess_input
elif model_type=="m":
    preproc = tf.keras.applications.mobilenet.preprocess_input
else:
    preproc = tf.keras.applications.densenet.preprocess_input

# Choose model
if model_type=="r":
    dec = tf.keras.applications.resnet.decode_predictions
elif model_type=="m":
    dec = tf.keras.applications.mobilenet.decode_predictions
else:
    dec = tf.keras.applications.densenet.decode_predictions

## Load Generated Data Set

In [ ]:
es = {'file_name': tf.TensorSpec(shape=(), dtype=tf.string, name=None),
 'image': tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None),
 'label': tf.TensorSpec(shape=(), dtype=tf.int64, name=None)}

mydataset = tf.data.experimental.load("../datasets/Imagenet/pruning/3kImages/",es).batch(20).prefetch(1)

## Evaluation Jobs

In [ ]:
# Specify which model(pruned/pruned+quantized) to evaluate
model_p_or_pqat = 'p' # p -> pruned model; qat -> pruned and quantized model

In [ ]:
if model_p_or_pqat == 'p':
    q_model = p_model
else:
    q_model = pqat_model

### DSSIM Data

For this job you must have dssim downloaded which can be done byusing their github link, https://github.com/kornelski/dssim.

In [ ]:
def job1(image,fil,og):
    
    tf.keras.preprocessing.image.save_img("./tmp/fake1.png", image)
    tf.keras.preprocessing.image.save_img("./tmp/real1.png", og)
    
    process = subprocess.run(['./path/to/dssim','./tmp/fake1.png','./tmp/real1.png'], 
                         stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE,
                         universal_newlines=True)
    if process.stdout.split('\t')[0] == '':
        print(process.stderr)
    else:
        dssim_data.append(process.stdout.split('\t')[0])

### Confidence Delta Calculation

In [ ]:
def job5(image, fil, og):
    
    image = np.copy(image)
    og = np.copy(og)
    
    real = np.expand_dims(og, axis=0)
    attack = np.expand_dims(image, axis=0)
    
    pred1, pred2= model.predict(real), q_model.predict(real)
    
    real_r_pred = dec(pred1, top=1000)
    real_q_pred = dec(pred2, top=1000)
    
    pred1, pred2= model.predict(attack), q_model.predict(attack)
    
    attack_r_pred = dec(pred1, top=1000)
    attack_q_pred = dec(pred2, top=1000)
    
    r_label = real_r_pred[0][0][1]
    
    print(r_label)
    
    rr = float(real_r_pred[0][0][2]) #Correct Label OG Model on OG image
    rq = float(real_q_pred[0][0][2]) #Correct Label PRuned Model on OG image
    
    for i in range(0,1000):
        if attack_q_pred[0][i][1] == r_label:
            aq = float(attack_q_pred[0][i][2])
            break
                       
    for i in range(0,1000):
        if attack_r_pred[0][i][1] == r_label:
            ar = float(attack_r_pred[0][i][2])
            break
    
    
    confRR_data.append(rr)
    confAR_data.append(ar)
    confRQ_data.append(rq)
    confAQ_data.append(aq)

### Evasion cost

In [ ]:
def job4(image,fil,og):
    
    image = np.copy(image)
    og = np.copy(og)
      
    real = np.expand_dims(preproc(og), axis=0)
    attack = np.expand_dims(preproc(image), axis=0)
    
    pred1, pred2= model.predict(real), q_model.predict(real)
    
    r_label = np.argmax(pred1)
    
    pred1, pred2= model.predict(attack), q_model.predict(attack)
    
    attack_r_pred = np.argmax(pred1)
    attack_q_pred = np.argmax(pred2)
    
    if (r_label == attack_r_pred and r_label == attack_q_pred):
        CC[0] += 1
        return
    if (r_label != attack_r_pred and r_label != attack_q_pred):
        WW[0] += 1
        return
    if (r_label != attack_r_pred and r_label == attack_q_pred):
        WC[0] += 1
        return
        
    print("Data might be damaged: ", r_label," ",attack_r_pred, " ",attack_q_pred)
    p[0]+=1
    if p[0]==2:
        assert(False)
    
    return
    

In [ ]:
'''
Data variables for evaluation.
'''

folderName= net + 'net_imagenet_images_second'
filterName= net +'net_imagenet_filters_second'
dataFolder= net +'net_imagenet_data_second'

failureFolderName='failure/' + folderName
failureFilterName='failure/' + filterName

dataName= 'second'

#Job1
dssim_data = []

#Job5
confRR_data = []
confAR_data = []
confRQ_data = []
confAQ_data = []

#Job4
CW = [0]
CC = [0]
WC = [0]
WW = [0]

## Perform Evaluation

In [ ]:
for i,filename in enumerate(os.listdir(folderName)):

    try:
        filename.index("@")
    except:
        print(filename)
        continue
        
    if filename.endswith('.npy'):
        count[0]+=1
        CW[0] += 1
        
        numbers = filename[6:].split('@')
        position = int(numbers[0])
        total = int(numbers[1][:-4])
        
        print(position)
        
        image = np.load(folderName+"/"+filename)
        fil = np.load(filterName+"/"+filename)
        og = image - fil
        
        
        job5(image,fil,og)
        
print(count[0])
print("Processing Failures")

for i,filename in enumerate(os.listdir(failureFolderName)):
    
    try:
        filename.index("@")
    except:
        print(filename)
        continue
        
    if filename.endswith('.npy'):
        count[0]+=1
        numbers = filename[6:].split('@')
        position = int(numbers[0])
        total = int(numbers[1][:-4])
        
        print(position)
        
        image = np.load(failureFolderName+"/"+filename)
        fil = np.load(failureFilterName+"/"+filename)
        og = image - fil
        
        job4(image,fil,og)
        job5(image,fil,og)

## Analysed and Saved Data

Insert name of data file stored here to be graphed and evaluated.

In [ ]:
textfile = open("./results/evaluation.csv", "w")

for element in confRR_data:
    textfile.write(str(element) + ", ")

textfile.write("\n")

for element in confRQ_data:
    textfile.write(str(element) + ", ")

textfile.write("\n")

for element in confAR_data:
    textfile.write(str(element) + ", ")

textfile.write("\n")

for element in confAQ_data:
    textfile.write(str(element) + ", ")

textfile.write("\n")

textfile.close()

In [ ]:
print(CC) # Both model predicted correctly
print(CW) # Only full-precision model predicted correctly
print(WC) # Only pruned/pruned+quantized model predicted correctly
print(WW) # Both model predicted incorrectly
print(CC[0]+WC[0]+CW[0]+WW[0])
print(count[0]/3000)

In [ ]:
dssim_data = np.array(dssim_data).astype(float)
np.mean(dssim_data),np.std(dssim_data),np.max(dssim_data),len(dssim_data)